Zero-shot (instruction-only)

In [ ]:
from openai import OpenAI
import os

client = OpenAI(base_url=os.environ["NIM_BASE_URL"], api_key=os.getenv("NIM_API_KEY","nvapi-..."))

def zero_shot(prompt: str, user_input: str):
    return client.chat.completions.create(
        model=os.environ["NIM_MODEL"],
        messages=[
            {"role":"system","content": prompt},
            {"role":"user","content": user_input},
        ],
        temperature=0.2,
        top_p=0.9,
        max_tokens=200
    ).choices[0].message.content


One-shot / Few-shot (demonstrations as message pairs)

In [ ]:
def few_shot(system_prompt: str, demos: list[tuple[str,str]], user_input: str):
    messages = [{"role":"system","content": system_prompt}]
    for x, y in demos:
        messages += [
            {"role":"user","content": x},
            {"role":"assistant","content": y},
        ]
    messages.append({"role":"user","content": user_input})

    return client.chat.completions.create(
        model=os.environ["NIM_MODEL"],
        messages=messages,
        temperature=0.2,
        top_p=0.9,
        max_tokens=200
    ).choices[0].message.content


Example demos:

In [ ]:
system_prompt = "Classify priority as P0/P1/P2/P3. Output only the label."

demos = [
  ("Service down for all users. No workaround.", "P0"),
  ("Intermittent errors for 10% users. Workaround exists.", "P1"),
  ("Minor UI bug. No impact on core flow.", "P3"),
]

print(few_shot(system_prompt, demos, "Checkout failures in one region, workaround: manual retry."))


Production Implementation: Dynamic Prompt Selector
Below is a Python conceptual implementation for an Enterprise Dynamic Few-Shot Selector. This is what you would build in a framework like LangChain or LlamaIndex.

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class DynamicFewShotSelector:
    def __init__(self, example_library, embedding_model):
        """
        example_library: List of dicts [{'input': '...', 'output': '...'}, ...]
        embedding_model: Function that returns vector for text
        """
        self.examples = example_library
        self.embedding_model = embedding_model
        # Pre-compute embeddings for the library (Cache this in production!)
        self.library_vectors = [self.embedding_model(ex['input']) for ex in example_library]

    def get_best_k_examples(self, user_query, k=3):
        """
        Selects top k examples semantically similar to the user query.
        """
        query_vector = self.embedding_model(user_query)
        
        # Calculate Cosine Similarity
        scores = cosine_similarity([query_vector], self.library_vectors)[0]
        
        # Get indices of top k scores
        top_indices = np.argsort(scores)[-k:][::-1]
        
        selected_examples = [self.examples[i] for i in top_indices]
        return selected_examples

    def construct_prompt(self, user_query, k=3):
        # 1. Retrieve
        shots = self.get_best_k_examples(user_query, k)
        
        # 2. Format (Enterprise Standard: ChatML or similar)
        prompt_text = "System: You are a helpful classifier.\n"
        
        # 3. Inject Few-Shot Examples
        for shot in shots:
            prompt_text += f"User: {shot['input']}\nAssistant: {shot['output']}\n"
            
        # 4. Append Actual Query
        prompt_text += f"User: {user_query}\nAssistant:"
        
        return prompt_text

# Usage Scenario:
# You have 1000 support tickets categorized.
# User asks a new question.
# We find the 3 most similar past tickets and feed them as few-shot examples.